In [ ]:
"""
Pipeline de ingestão RAG com CHUNKING HÍBRIDO (profissional)
===========================================================

Chunking híbrido = padrão real de produção.

Fluxo:
1) Leitura de PDF / DOCX
2) Pré-chunk determinístico (controle de tamanho)
3) Refinamento semântico com LLM (chunk <= 1000 chars)
4) Documents + metadados
5) Embeddings
6) FAISS persistido em disco

Este modelo resolve:
✔ PDFs longos (artigos científicos)
✔ Controle rígido de tamanho
✔ Qualidade semântica
✔ Escalabilidade

Python recomendado: 3.11
"""

# ============================================================
# 1. IMPORTS
# ============================================================
import os

from pypdf import PdfReader
from docx import Document as DocxDocument

from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.documents import Document
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

from langchain_ollama import ChatOllama
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS


# ============================================================
# 2. LEITURA DE ARQUIVOS
# ============================================================

def load_pdf(path: str) -> str:
    reader = PdfReader(path)
    text = ""
    for page in reader.pages:
        page_text = page.extract_text()
        if page_text:
            text += page_text + "\n"
    return text


def load_docx(path: str) -> str:
    doc = DocxDocument(path)
    return "\n".join(p.text for p in doc.paragraphs if p.text.strip())


# ============================================================
# 3. PRÉ-CHUNKING DETERMINÍSTICO (OBRIGATÓRIO)
# ============================================================

# Pré-chunks maiores para o LLM trabalhar localmente
PRE_CHUNK_SIZE = 3000
PRE_CHUNK_OVERLAP = 300

pre_splitter = RecursiveCharacterTextSplitter(
    chunk_size=PRE_CHUNK_SIZE,
    chunk_overlap=PRE_CHUNK_OVERLAP,
    separators=["\n\n", "\n", ". ", " ", ""],
)


# ============================================================
# 4. LLM PARA REFINAMENTO SEMÂNTICO
# ============================================================

llm = ChatOllama(
    base_url="http://localhost:11434",
    model="gemma3:4b",
    temperature=0.0,
)

MAX_FINAL_CHARS = 1000

chunking_prompt = ChatPromptTemplate.from_template(
    """
Você é um sistema automático de segmentação de texto.

REGRAS OBRIGATÓRIAS:
- NÃO explique
- NÃO reescreva
- NÃO acrescente texto novo
- APENAS copie trechos do texto original
- Cada chunk deve ter NO MÁXIMO 1000 caracteres
- Preserve a ordem original
- Use <CHUNK> como delimitador

Texto:
{text}
"""
)

chunking_chain = chunking_prompt | llm | StrOutputParser()


def llm_refine_chunks(text: str) -> list[str]:
    """
    Refina um pré-chunk usando LLM e garante
    limite rígido de tamanho.
    """
    response = chunking_chain.invoke({"text": text})

    raw_chunks = [
        c.strip() for c in response.split("<CHUNK>") if c.strip()
    ]

    safe_chunks = []
    for chunk in raw_chunks:
        if len(chunk) > MAX_FINAL_CHARS:
            chunk = chunk[:MAX_FINAL_CHARS]
        safe_chunks.append(chunk)

    return safe_chunks


# ============================================================
# 5. CARREGAMENTO DOS DOCUMENTOS
# ============================================================

FILES = []

if os.path.exists("datasets/tattoo.pdf"):
    FILES.append(("pdf", "tattoo.pdf", load_pdf("datasets/tattoo.pdf")))

#if os.path.exists("documento.docx"):
#    FILES.append(("docx", "documento.docx", load_docx("documento.docx")))

if not FILES:
    raise FileNotFoundError("Nenhum PDF ou DOCX encontrado.")


# ============================================================
# 6. CHUNKING HÍBRIDO (CORE DO PIPELINE)
# ============================================================

documents = []

for file_type, path, text in FILES:
    # 1️⃣ Pré-chunk determinístico
    pre_chunks = pre_splitter.split_text(text)

    # 2️⃣ Refinamento semântico por LLM
    final_chunks = []
    for pre_chunk in pre_chunks:
        refined = llm_refine_chunks(pre_chunk)
        final_chunks.extend(refined)

    # 3️⃣ Criar Documents
    for i, chunk in enumerate(final_chunks):
        documents.append(
            Document(
                page_content=chunk,
                metadata={
                    "source": os.path.basename(path),
                    "file_type": file_type,
                    "chunk_id": i,
                    "chunking": "hybrid",
                },
            )
        )

print(f"Total de chunks híbridos gerados: {len(documents)}")


# ============================================================
# 7. EMBEDDINGS
# ============================================================

embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2",
    model_kwargs={"device": "cpu"},
    encode_kwargs={"normalize_embeddings": True},
)


# ============================================================
# 8. VECTORSTORE FAISS
# ============================================================

vectorstore = FAISS.from_documents(
    documents=documents,
    embedding=embeddings,
)

print("Vector store FAISS criado com sucesso.")


# ============================================================
# 9. SALVAR VECTORSTORE EM DISCO
# ============================================================

VECTORSTORE_DIR = "vectorstore_faiss"
os.makedirs(VECTORSTORE_DIR, exist_ok=True)

vectorstore.save_local(VECTORSTORE_DIR)

print(f"Vector store salvo em: {VECTORSTORE_DIR}")


# ============================================================
# 10. CONCLUSÃO PROFISSIONAL
# ============================================================

"""
Este é o PADRÃO CORRETO de chunking em RAG sério:

✔ Pré-chunk determinístico (controle)
✔ LLM apenas para refinamento local
✔ Limites respeitados
✔ PDFs longos funcionam corretamente
✔ Pronto para avaliação e produção

Qualquer RAG robusto segue esta arquitetura.
"""


In [2]:
# ============================================================
# INSPECIONAR CHUNKS ARMAZENADOS NO VECTORSTORE FAISS
# ============================================================

# Objetivo:
# - Verificar quais chunks foram realmente salvos no FAISS
# - Inspecionar texto + metadados
# - Fazer debug de ingestão (etapa crítica em RAG)

# Premissas:
# - vectorstore já foi criado OU carregado do disco
# - embeddings compatíveis já estão configurados


# ------------------------------------------------------------
# 1. ACESSAR DOCUMENTOS INTERNOS DO VECTORSTORE
# ------------------------------------------------------------
# O FAISS guarda os Documents em um docstore interno

docstore = vectorstore.docstore
doc_ids = list(docstore._dict.keys())

print(f"Total de chunks no vectorstore: {len(doc_ids)}")


# ------------------------------------------------------------
# 2. VISUALIZAR ALGUNS CHUNKS (RECOMENDADO)
# ------------------------------------------------------------
# Evita poluir o terminal com texto demais

for doc_id in doc_ids[:5]:  # mostra apenas os 5 primeiros
    doc = docstore._dict[doc_id]

    print("=" * 80)
    print(f"Doc ID: {doc_id}")
    print(f"Source: {doc.metadata.get('source')}")
    print(f"File type: {doc.metadata.get('file_type')}")
    print(f"Chunk ID: {doc.metadata.get('chunk_id')}")
    print(f"Chunking: {doc.metadata.get('chunking')}")
    print(f"Tamanho (caracteres): {len(doc.page_content)}")
    print("-" * 80)
    print(doc.page_content[:600])  # limita o texto exibido
    print("...\n")


# ------------------------------------------------------------
# 3. INSPEÇÃO COMPLETA (USAR COM CUIDADO)
# ------------------------------------------------------------
# Se você realmente quiser ver TODOS os chunks:

# for doc_id in doc_ids:
#     doc = docstore._dict[doc_id]
#     print(doc.page_content)


# ============================================================
# OBSERVAÇÃO PROFISSIONAL IMPORTANTE
# ============================================================

"""
Boas práticas ao inspecionar chunks:

✔ Sempre verifique os chunks ANTES de culpar o modelo
✔ Confira se não há chunks vazios ou duplicados
✔ Verifique tamanhos (<= 1000 chars)
✔ Confirme metadados (source, chunk_id)

90% dos problemas de RAG vêm de ingestão mal validada.
"""


Total de chunks no vectorstore: 1
Doc ID: 173bfcf2-c083-483f-8782-59f23813e765
Source: tattoo.pdf
File type: pdf
Chunk ID: 0
Chunking: llm_based
Tamanho (caracteres): 1000
--------------------------------------------------------------------------------
Okay, here's a breakdown of the provided text, categorized for clarity and with some potential areas for further exploration:

**1. Author Information:**

*   **Anderson Brilhador:**
    *   Degree: MSc in Computer Science
    *   Institution: Federal University of Technology - Paraná (UTFPR)
    *   Research Interests: Computer vision, machine learning, deep learning, data mining.
*   **Rodrigo Tchalski da Silva Va:**
    *   Degree: MSc in Industrial Computing
    *   Institution: UTFPR
    *   Research Interests: Computer vision, tattoo recognition, machine learning, and complex networks.
*
...



'\nBoas práticas ao inspecionar chunks:\n\n✔ Sempre verifique os chunks ANTES de culpar o modelo\n✔ Confira se não há chunks vazios ou duplicados\n✔ Verifique tamanhos (<= 1000 chars)\n✔ Confirme metadados (source, chunk_id)\n\n90% dos problemas de RAG vêm de ingestão mal validada.\n'

In [ ]:
# ============================================================
# SALVAR E RECARREGAR VECTORSTORE FAISS EM ARQUIVO
# ============================================================

# Objetivo:
# - Persistir o índice FAISS em disco
# - Evitar reprocessar embeddings a cada execução
# - Usar padrão profissional de ingestão RAG

# Premissas:
# - `documents` já existe (chunks convertidos em Document)
# - `embeddings` já está configurado
# - FAISS está sendo usado como vectorstore

from langchain_community.vectorstores import FAISS
import os


# ------------------------------------------------------------
# 1. DEFINIR DIRETÓRIO DE PERSISTÊNCIA
# ------------------------------------------------------------
# Esse diretório armazenará:
# - index.faiss  -> índice vetorial
# - index.pkl    -> metadados (Document, embeddings, etc.)

VECTORSTORE_DIR = "vectorstore_faiss"

os.makedirs(VECTORSTORE_DIR, exist_ok=True)


# ------------------------------------------------------------
# 2. CRIAR VECTORSTORE A PARTIR DOS DOCUMENTS
# ------------------------------------------------------------

vectorstore = FAISS.from_documents(
    documents=documents,
    embedding=embeddings,
)

print("Vector store FAISS criado com sucesso.")


# ------------------------------------------------------------
# 3. SALVAR VECTORSTORE EM DISCO
# ------------------------------------------------------------
# Este é o passo-chave da persistência

vectorstore.save_local(VECTORSTORE_DIR)

print(f"Vector store FAISS salvo em: {VECTORSTORE_DIR}")


# ============================================================
# (EXECUÇÕES FUTURAS) — COMO RECARREGAR O VECTORSTORE
# ============================================================

# Em um novo script ou nova execução, NÃO gere embeddings novamente.
# Apenas recarregue o índice salvo.

# IMPORTANTE:
# - embeddings DEVEM ser do mesmo modelo usado na criação

vectorstore = FAISS.load_local(
    VECTORSTORE_DIR,
    embeddings,
    allow_dangerous_deserialization=True,  # necessário por usar pickle
)

print("Vector store FAISS carregado do disco com sucesso.")


# ------------------------------------------------------------
# 4. CRIAR RETRIEVER A PARTIR DO VECTORSTORE RECARREGADO
# ------------------------------------------------------------

retriever = vectorstore.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 3},
)


# ------------------------------------------------------------
# 5. TESTE DE SANIDADE (OPCIONAL)
# ------------------------------------------------------------

query = "O que é RAG?"
docs = retriever.invoke(query)

for doc in docs:
    print("=" * 80)
    print(f"Chunk ID: {doc.metadata.get('chunk_id')}")
    print(f"Source: {doc.metadata.get('source')}")
    print("-" * 80)
    print(doc.page_content[:400])
    print()


# ============================================================
# OBSERVAÇÃO PROFISSIONAL FINAL
# ============================================================

"""
Boas práticas que ESTE código segue:

✔ Embeddings gerados apenas uma vez
✔ Vectorstore persistido em disco
✔ Reuso rápido e barato do índice
✔ Separação clara entre ingestão e consulta

Arquitetura recomendada em produção:

ingest.py   -> cria chunks, embeddings, salva FAISS
rag.py      -> carrega FAISS e responde perguntas
api.py      -> expõe o RAG via FastAPI

Isso é RAG em nível profissional.
"""
